In [1]:
import pandas as pd 
import numpy as np
import os
#import cv2
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, Flatten, Convolution2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.metrics import Recall as recall
from keras import layers, models

import matplotlib.pyplot as plt
%matplotlib inline

### Preprocessing

In [2]:
train_path = "data/chest_xray/train"
test_path = "data/chest_xray/test"
val_path = "data/chest_xray/val"

In [3]:
# 
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [4]:
# 
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(200,200),
    batch_size = 5216,#32
    color_mode = 'grayscale',
    class_mode = 'categorical')

test_generator = test_datagen.flow_from_directory(
    test_path, 
    target_size = (200, 200),
    batch_size = 624, #32
    color_mode = 'grayscale',
    class_mode = 'categorical')

val_generator = test_datagen.flow_from_directory(
    val_path, 
    target_size = (200, 200),
    batch_size = 16, #32
    color_mode = 'grayscale',
    class_mode = 'categorical')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [5]:
#train_class_names = list(train_generator.class_indices.keys())
#train_class_names

In [6]:
# create the data sets
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)
val_images, val_labels = next(val_generator)

In [7]:
# Explore the dataset
m_train = train_images.shape[0]
num_px = train_images.shape[1]
m_test = test_images.shape[0]
m_val = val_images.shape[0]

print ("Number of training samples: " + str(m_train))
print ("Number of testing samples: " + str(m_test))
print ("Number of validation samples: " + str(m_val))
#print ("train_images shape: " + str(train_images.shape))
#print ("train_labels shape: " + str(train_labels.shape))
#print ("test_images shape: " + str(test_images.shape))
#print ("test_labels shape: " + str(test_labels.shape))
#print ("val_images shape: " + str(val_images.shape))
#print ("val_labels shape: " + str(val_labels.shape))

Number of training samples: 5216
Number of testing samples: 624
Number of validation samples: 16


In [8]:
train_img = train_images.reshape(train_images.shape[0], -1)
test_img = test_images.reshape(test_images.shape[0], -1)
val_img = val_images.reshape(val_images.shape[0], -1)

print(train_img.shape)
print(test_img.shape)
print(val_img.shape)

(5216, 40000)
(624, 40000)
(16, 40000)


In [10]:
train_y = np.reshape(train_labels[:,0], (5216,1))
test_y = np.reshape(test_labels[:,0], (624,1))
val_y = np.reshape(val_labels[:,0], (16,1))

In [11]:
model = models.Sequential()
model.add(Dense(20, activation='relu', input_shape = (40000,)))
model.add(Dense(7, activation = 'relu'))
model.add(Dense(5, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

Metal device set to: Apple M1 Max


2022-12-06 00:28:04.053382: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-06 00:28:04.053901: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [12]:
model.compile(optimizer = 'sgd',
             loss = 'binary_crossentropy',
             metrics = [recall()])

history = model.fit(train_img,
                   train_y,
                   epochs = 50,
                   batch_size = 32,
                   validation_data = (val_img, val_y))

Epoch 1/50


2022-12-06 00:28:16.483254: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-06 00:28:16.607925: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


163/163 [==============================] - 2s 13ms/step - loss: 0.5737 - recall: 0.0149 - val_loss: 0.8804 - val_recall: 0.0000e+00
Epoch 2/50
  6/163 [>.............................] - ETA: 1s - loss: 0.5365 - recall: 0.0000e+00

2022-12-06 00:28:18.780249: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


163/163 [==============================] - 2s 12ms/step - loss: 0.4510 - recall: 0.3609 - val_loss: 0.5876 - val_recall: 0.8750
Epoch 3/50
163/163 [==============================] - 2s 12ms/step - loss: 0.3846 - recall: 0.6003 - val_loss: 0.6242 - val_recall: 0.3750
Epoch 4/50
163/163 [==============================] - 2s 12ms/step - loss: 0.3526 - recall: 0.6764 - val_loss: 0.5898 - val_recall: 0.8750
Epoch 5/50
163/163 [==============================] - 2s 11ms/step - loss: 0.3453 - recall: 0.6957 - val_loss: 0.6326 - val_recall: 0.5000
Epoch 6/50
163/163 [==============================] - 2s 11ms/step - loss: 0.3285 - recall: 0.7360 - val_loss: 0.5639 - val_recall: 0.7500
Epoch 7/50
163/163 [==============================] - 2s 12ms/step - loss: 0.3141 - recall: 0.7442 - val_loss: 0.8234 - val_recall: 0.2500
Epoch 8/50
163/163 [==============================] - 2s 12ms/step - loss: 0.3162 - recall: 0.7509 - val_loss: 0.5961 - val_recall: 0.6250
Epoch 9/50
163/163 [==================

In [13]:
results_train = model.evaluate(train_img, train_y)
results_train

163/163 [==============================] - 2s 12ms/step - loss: 0.2436 - recall: 0.9418


[0.24358834326267242, 0.9418344497680664]

In [14]:
results_test = model.evaluate(test_img, test_y)
results_test

20/20 [==============================] - 0s 10ms/step - loss: 0.3399 - recall: 0.8376


[0.3399181663990021, 0.8376069068908691]

### Base Model - CNN

In [21]:
# Initializing the CNN
base_model = Sequential()

# 1st convolution layer and pooling
base_model.add(layers.Conv2D(32, (3, 3), input_shape=(200, 200, 1), activation = 'relu'))
#base_model.add(layers.Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
base_model.add(MaxPooling2D(pool_size = (3, 3)))

# 2nd convolution layer and pooling
base_model.add(layers.Conv2D(32, (3, 3), activation = 'relu'))
base_model.add(MaxPooling2D(pool_size = (3, 3)))

# 3rd 
base_model.add(layers.Conv2D(64, (3,3), activation ='relu'))
base_model.add(MaxPooling2D(pool_size = (3, 3)))

# plattening the layers
base_model.add(Flatten())

# adding the fully connected dense layer
base_model.add(Dense(64, activation = 'relu'))
base_model.add(Dropout(0.3))
# output layer
base_model.add(Dense(1, activation = 'sigmoid')) # is it 1 or 2 for binary classification?

# compiler
base_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=[recall()])

history = base_model.fit(train_images,
                        train_y, # changed train_generator and added the train_y
                        batch_size = 32,
                        epochs = 50,
                        validation_data = (val_images, val_y))
                        #validation_data = (val_generator))

# dropouts, keep it closer to the cnn , 1-2nd 

Epoch 1/50


2022-12-06 11:27:42.106253: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


163/163 [==============================] - 4s 22ms/step - loss: 0.4575 - recall_3: 0.3184 - val_loss: 1.0317 - val_recall_3: 0.2500
Epoch 2/50
  4/163 [..............................] - ETA: 3s - loss: 0.3220 - recall_3: 0.5484

2022-12-06 11:27:45.726989: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


163/163 [==============================] - 3s 19ms/step - loss: 0.2521 - recall_3: 0.7785 - val_loss: 0.9188 - val_recall_3: 0.2500
Epoch 3/50
163/163 [==============================] - 3s 19ms/step - loss: 0.1969 - recall_3: 0.8471 - val_loss: 0.6485 - val_recall_3: 0.2500
Epoch 4/50
163/163 [==============================] - 3s 19ms/step - loss: 0.1846 - recall_3: 0.8635 - val_loss: 1.1703 - val_recall_3: 0.3750
Epoch 5/50
163/163 [==============================] - 3s 19ms/step - loss: 0.1591 - recall_3: 0.8889 - val_loss: 1.0600 - val_recall_3: 0.2500
Epoch 6/50
163/163 [==============================] - 3s 19ms/step - loss: 0.1408 - recall_3: 0.9068 - val_loss: 1.0165 - val_recall_3: 0.2500
Epoch 7/50
163/163 [==============================] - 3s 19ms/step - loss: 0.1378 - recall_3: 0.9031 - val_loss: 1.2947 - val_recall_3: 0.2500
Epoch 8/50
163/163 [==============================] - 3s 20ms/step - loss: 0.1292 - recall_3: 0.9239 - val_loss: 1.3534 - val_recall_3: 0.2500
Epoch 9/50

In [22]:
test_y.shape

(624, 1)

In [23]:
test_images.shape

(624, 200, 200, 1)

In [28]:
train_y.shape

(5216, 1)

In [29]:
train_images.shape

(5216, 200, 200, 1)

In [24]:
results_train = base_model.evaluate(train_images, train_y)
results_train

163/163 [==============================] - 2s 11ms/step - loss: 0.0022 - recall_3: 1.0000


[0.0022304807789623737, 1.0]

In [25]:
results_test = model.evaluate(test_images, test_y)
results_test

InvalidArgumentError: Graph execution error:

Detected at node 'sequential/dense/MatMul' defined at (most recent call last):
    File "/Users/Viki/miniforge3/envs/tf_env/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/Viki/miniforge3/envs/tf_env/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/Users/Viki/miniforge3/envs/tf_env/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/Viki/miniforge3/envs/tf_env/lib/python3.10/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/Users/Viki/miniforge3/envs/tf_env/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/Users/Viki/miniforge3/envs/tf_env/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.r

In [ ]:
# recall on the validation set it tells that the model is probably a bit overfitting, but as the validation set is too small to say it yet if it is overfitting.
# So as a next step I will try to play with the regularization parameters.



In [ ]:
base_model.summary()

In [ ]:
# 

### Data Augmentation

In [ ]:
# define the stakeholders 